## Table of Contents:
* [Introduction](#Introduction)
* [Import Libraries](#import-libraries)
* [Unpickling the dictionary](#unpickle)
* [How Stock price relates to exogenous variables](#best)
* [High momentum stock list](#momentum)


## Introduction <a class="anchor" id="Introduction"></a>

## Exploratory Data Analysis

In this notebook, we have carried out exploratory data analysis of stock price data. We have explore the effect of exogenous variables 
on stock price for eack of the company. Finally, we have pulled high momentum stocks for one, five and ten years investment.

## Importing the necessary libraries <a class="anchor" id="import-libraries"></a>

In [29]:
import pandas as pd
import pandas
import datetime as dt
from datetime import datetime
from urllib.error import HTTPError
#import requests
#from bs4 import BeautifulSoup
import re
import csv
import numpy as np
import warnings
from sklearn import datasets, linear_model, metrics
import pickle
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import statsmodels.api as sm
from scipy import stats
from sklearn.model_selection import KFold 
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso

## Unpickling the data<a class="anchor" id="unpickle"></a>

In [50]:
with open('exo_df.pickle', 'rb') as handle:
    exo_df = pickle.load(handle)

with open('cons_cyc_df_years.pickle', 'rb') as handle:
    cons_cyc_df_years = pickle.load(handle)
   


## How Stock price relates to exogenous variables<a class="anchor" id="best"></a>


In [52]:
cmpny_model = {}
missing_revenue = []

warnings.filterwarnings("ignore", category=FutureWarning)
def best_model(company_dict):
    kf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=1)
    for keys in company_dict:
        company = company_dict[keys].drop(['Interest Rate (%)'], axis = 1)
        
        X = company[company.columns[1:4]].values#.reshape(-1,1)
        y = company[company.columns[0]].values#.reshape(-1,1)
        


        try:
        #Linearregression
            model_lin = LinearRegression()
            Linear_reg_CV_score = np.mean(cross_val_score(model_lin, X, y, scoring='r2', cv=kf))
            
        #Ridgeregression
            #ridge = RidgeCV(cv=10).fit(X, y)
            #Ridge_reg_CV_score = ridge.score(X, y)
            model_ridge = Ridge()
            Ridge_reg_CV_score = np.mean(cross_val_score(model_ridge, X, y, scoring='r2', cv=kf))
        #Lassoregression
            #lasso = LassoCV(cv=10).fit(X, y)
            #Lasso_reg_CV_score = lasso.score(X, y)
            model_lasso = Lasso()
            Lasso_reg_CV_score = np.mean(cross_val_score(model_lasso, X, y, scoring='r2', cv=kf))
        
        #polynomialregression
            poly = PolynomialFeatures(degree=2, include_bias=True)
            x_train_trans = poly.fit_transform(X)
            PolyLinear_reg_CV_score  = np.mean(cross_val_score(model_lin, x_train_trans, y, cv=kf, scoring='r2'))
        
        except ValueError:
            missing_revenue.append(keys)
            
        CV_scores = {'Linear Regression': Linear_reg_CV_score, 'Ridge Regression': Ridge_reg_CV_score,'Lasso Regression': Lasso_reg_CV_score,'Polynomial Regression':PolyLinear_reg_CV_score}
        #print (CV_scores)
        max_score = 0
        best_model = None
        for models in CV_scores:
            if (max_score < CV_scores[models]):
                max_score = CV_scores[models]
                best_model = models
        
        cmpny_model[keys] = (max_score, best_model)
      
    return cmpny_model
        

#best_model_dict = best_model(exo_df)

#print (best_model_dict)


score_list_dict = {}
for keys in best_model_dict:
    if best_model_dict[keys][0] not in score_list_dict:
        score_list_dict[best_model_dict[keys][0]] = [keys]
    else:
        score_list_dict[best_model_dict[keys][0]].append(keys) 

final_model_dict = {}
cols = ['Company', 'Revenue(Million $)', 'PE Ratio']

def nameToModelLookup(modelName):
    if method == 'Lasso Regression':
        return Lasso()
    if method == 'Ridge Regression':
        return Ridge()
    return None

for keys in sorted(score_list_dict.keys(), reverse = True)[:25]:
    company, method = ((score_list_dict[keys][0]), best_model_dict[score_list_dict[keys][0]][1])
    X = exo_df[company][exo_df[company].columns[1:3]]
    y = exo_df[company][exo_df[company].columns[0]]
    if nameToModelLookup(method):
        if not method in final_model_dict:
            final_model_dict[method] = pd.DataFrame(columns = cols)
        model = nameToModelLookup(method)
        model.fit(X,y)
        coef_val = model.coef_
        final_model_dict[method] = final_model_dict[method].append({'Company':company,'Revenue(Million $)': coef_val[0],
                                                'PE Ratio' : coef_val[1]}, ignore_index = True)
    
            
final_model_dict 

    

    

    

    
    

  
    
    



{'Ridge Regression':    Company Revenue(Million $)  PE Ratio
 0     ROST           0.030573  0.968508
 1     TXRH           0.099845  0.488046
 2     SBUX           0.015331  0.650453
 3      GPC           0.031327  2.644666
 4      KMX           0.017691  1.444918
 5       AN           0.016532  2.223631
 6      PAG           0.011968  1.623594
 7     CULP           0.510145  0.990411
 8     CBRL           0.548214  5.471312
 9       HD           0.009753  5.413046
 10     SCI           0.085661 -0.200098,
 'Lasso Regression':    Company Revenue(Million $)  PE Ratio
 0     DORM           0.452424  0.597425
 1      NKE           0.009056  1.042811
 2      PKG            0.05766  1.601836
 3      ROL           0.111262  0.627605
 4     GNTX           0.044948  0.163265
 5     EXPE            0.05944 -0.071147
 6     PZZA           0.293824  0.772794
 7     AMZN           0.022075  0.014554
 8      CSV           1.016403  0.135778
 9      LAD           0.045217  0.228626
 10    LCII     

## High momentum stock list <a class="anchor" id="momentum"></a>

In [19]:
hqm_ind = ['ticker',
        "one year price return",
        "one year return percentile",
        "five year price return",
        "five year return percentile",
        "ten year price return",
        "ten year return percentile",
          ]

year_list = ['one', 'five', 'ten']
hqm_score = pd.DataFrame(columns = hqm_ind)
for keys in cons_cyc_df_years:
    high_momntm_df = cons_cyc_df_years[keys].loc[:,(cons_cyc_df_years[keys].columns == 'Close')]
    one_year_return = high_momntm_df.asfreq('1Y', method = 'ffill').pct_change().dropna().mean().squeeze()
    five_year_return = high_momntm_df.asfreq('5Y', method = 'ffill').pct_change().dropna().mean().squeeze()
    ten_year_return = high_momntm_df.asfreq('10Y', method = 'ffill').pct_change().dropna().mean().squeeze()
    hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile":None}, ignore_index = True)


for rows in hqm_score.index:
    for year in year_list:
        hqm_score.loc[rows,'{} year return percentile'.format(year)] = stats.percentileofscore(hqm_score['{} year price return'.format(year)], 
                                                                              hqm_score.loc[rows,'{} year price return'.format(year)]) 

#alternate way to calculate percentile   
#hqm_score['one year return percentile'] = hqm_score['one year price return'].rank(pct = True)



hqm_score

/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_1688/238161240.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile":None}, ignore_index = True)
/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_1688/238161240.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile

/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_1688/238161240.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile":None}, ignore_index = True)
/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_1688/238161240.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile

/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_1688/238161240.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile":None}, ignore_index = True)
/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_1688/238161240.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile

/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_1688/238161240.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile":None}, ignore_index = True)
/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_1688/238161240.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile

/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_1688/238161240.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile":None}, ignore_index = True)
/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_1688/238161240.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile

/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_1688/238161240.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile":None}, ignore_index = True)
/var/folders/q8/h0qxrk9104304kbtwfz3q8r80000gn/T/ipykernel_1688/238161240.py:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  hqm_score = hqm_score.append({'ticker':keys, "one year price return":one_year_return,"one year return percentile": None, "five year price return": five_year_return, "five year return percentile" :None,"ten year price return":ten_year_return, "ten year return percentile

,ticker,one year price return,one year return percentile,five year price return,five year return percentile,ten year price return,ten year return percentile
0,AAN,0.082398,19.607843,0.304979,33.333333,0.695209,47.058824
1,ANF,-0.051728,2.614379,-0.50535,0.653595,-0.800787,1.960784
2,AAP,0.19377,64.705882,1.21039,78.431373,3.825812,89.542484
3,AIN,0.124032,35.294118,0.462518,39.869281,0.585392,43.137255
4,AMZN,0.492866,97.385621,3.359363,99.346405,18.003294,100.0
...,...,...,...,...,...,...,...
148,URBN,0.107913,28.104575,0.115041,18.954248,0.236648,32.679739
149,MTN,0.197819,67.320261,1.561267,85.620915,2.980602,84.313725
150,WWW,0.093025,22.875817,0.282632,30.718954,0.644867,45.098039
151,WYNN,0.160132,48.366013,0.22841,24.836601,0.440603,37.908497


In [13]:
hqm_score.sort_values('one year return percentile', ascending = False, ignore_index = True)[:10]

,ticker,one year price return,one year return percentile,five year price return,five year return percentile,ten year price return,ten year return percentile
0,KIRK,0.695917,100.0,0.942468,69.281046,2.273326,74.509804
1,TEN,0.618548,99.346405,0.651224,52.287582,1.527101,65.359477
2,CULP,0.553127,98.69281,2.297690,96.732026,7.174029,96.732026
3,MED,0.500701,98.039216,1.123123,76.470588,2.443469,78.431373
4,AMZN,0.492866,97.385621,3.359363,99.346405,18.003294,100.0
5,ESCA,0.440976,96.732026,1.025111,73.856209,0.581413,42.48366
6,RICK,0.419624,96.078431,0.638352,50.980392,1.520094,64.705882
7,LZB,0.408867,95.424837,0.872221,64.705882,1.749440,67.973856
8,DPZ,0.399587,94.771242,2.699532,98.69281,10.065230,98.69281
9,EXPE,0.392397,94.117647,1.724262,89.542484,4.663614,91.503268
